In [402]:
import cv2 as cv
import mrob
import numpy as np
import open3d as o3d
import os
import pickle

In [403]:
def pointcloudify_depth(depth, intrinsics, dist_coeff, undistort=True):
    shape = depth.shape[::-1]
    
    if undistort:
        undist_intrinsics, _ = cv.getOptimalNewCameraMatrix(intrinsics, dist_coeff, shape, 1, shape)
        inv_undist_intrinsics = np.linalg.inv(undist_intrinsics)

    else:
        inv_undist_intrinsics = np.linalg.inv(intrinsics)

    if undistort:
        # undist_depthi = cv.undistort(depthi, intrinsics, dist_coeff, None, undist_intrinsics)
        map_x, map_y = cv.initUndistortRectifyMap(intrinsics, dist_coeff, None
                                                  , undist_intrinsics, shape, cv.CV_32FC1)
        undist_depth = cv.remap(depth, map_x, map_y, cv.INTER_NEAREST)

    # Generate x,y grid for H x W image
    grid_x, grid_y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    grid = np.concatenate([np.expand_dims(grid_x, -1),
                           np.expand_dims(grid_y, -1)], axis=-1)

    grid = np.concatenate([grid, np.ones((shape[1], shape[0], 1))], axis=-1)

    # To normalized image coordinates
    local_grid = inv_undist_intrinsics @ grid.reshape(-1, 3).transpose()  # 3 x H * W

    # Raise by undistorted depth value from image plane to local camera space
    if undistort:
        local_grid = local_grid.transpose() * np.expand_dims(undist_depth.reshape(-1), axis=-1)

    else:
        local_grid = local_grid.transpose() * np.expand_dims(depth.reshape(-1), axis=-1)
        
    return local_grid.astype(np.float32)

In [560]:
filepath = '2021-04-01-15-49-41-traj.txt'
poses_quat = []
with open(filepath, 'r') as file:
    for line in file:
        poses_quat.append([float(i) for i in line.split(' ')])

In [561]:
azure_depth_folder = '../2021-04-01-15-49-41/_azure_depth_image_raw/'

depth_ts = np.array([int(file.split('.')[0]) for file in os.listdir(azure_depth_folder) 
                     if file.endswith('.npy')])
depth_ts.sort()
depth_ts

array([606624793570, 606824793570, 607024793570, 607224893570,
       607424904570, 607624893570, 607824893570, 608024893570,
       608224893570, 608424893570, 608624904570, 608824893570,
       609024893570, 609224893570, 609424893570, 609624782570,
       609824793570, 610024782570, 610224893570, 610424893570,
       610624904570, 610824904570, 611024893570, 611224893570,
       611424893570, 611624893570, 611824893570, 612024782570,
       612224893570, 612424904570, 612624893570, 612824893570,
       613024893570, 613224893570, 613424793570, 613624793570,
       613824893570, 614024893570, 614224893570, 614424893570,
       614624904570, 614824893570, 615024893570, 615224904570,
       615424893570, 615624893570, 615824782570, 616024904570,
       616224904570, 616424893570, 616624893570, 616824893570,
       617024904570, 617224793570, 617424793570, 617624782570,
       617824782570, 618024782570, 618224893570, 618424904570,
       618624893570, 618824893570, 619024893570, 619224

In [562]:
SETUP_CONFIG = 'bandeja_standard.pickle'
with open(SETUP_CONFIG, 'rb') as config:
    config_dict = pickle.load(config)

In [563]:
import numpy as np
import colorsys

def get_colors(num_colors):
    colors=[]
    for i in np.arange(0., 360., 360. / num_colors):
        hue = i/360.
        lightness = (50 + np.random.rand() * 10)/100.
        saturation = (90 + np.random.rand() * 10)/100.
        colors.append(colorsys.hls_to_rgb(hue, lightness, saturation))
    return colors

In [564]:
N = 20

colors = get_colors(len(poses_quat) // N + 1)
geometries = []
Ts = []
part = 120

for i, pose in enumerate(poses_quat[:part]): 
    t = pose[1:4]
    R = mrob.geometry.quat_to_so3(pose[4:8])
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    Ts.append(T)
    if not i % N:
        coord = o3d.geometry.TriangleMesh.create_coordinate_frame(0.1).transform(T)
        c = i / (part + 1)
        coord.paint_uniform_color([0.5,1 - c, (1 - c) / 2])
        geometries.append(coord)


T = np.load('azure2s10_standard_extrinsics.npy')
full_pcd = None
for i, ts in enumerate(depth_ts[:part]):
    if not i % N:
        depth = np.load(os.path.join(azure_depth_folder, str(ts) + '.npy'))
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pointcloudify_depth(depth, config_dict['depth']['dist_mtx'],
                                                config_dict['depth']['dist_coef']))
        pc = pcd.transform(T).transform(Ts[i])
#         pc.paint_uniform_color(colors[i // N])
        c = i / (part + 1)
        pc.paint_uniform_color([0.5,1 - c, (1 - c)/ 2])
        if full_pcd == None:
            full_pcd = pc
        else:
            full_pcd += pc

downpcd = full_pcd.voxel_down_sample(voxel_size=0.01)
geometries.append(downpcd)

o3d.visualization.draw_geometries(geometries)